In [ ]:
import os
if False:
    PRIME_DATASET_FOLDER_PATH = "./DataPlaceholder"

    path = "/DataPlaceholder"
    mriImages = os.listdir("./DataPlaceholder/MRI")
    ctImages = os.listdir("./DataPlaceholder/CT")

    MRI_FOLDER_NAME = "MRIWIDENED"
    CT_FOLDER_NAME = "CTWIDENED"

    mri_base_path = os.path.join(PRIME_DATASET_FOLDER_PATH, MRI_FOLDER_NAME)
    ct_base_path = os.path.join(PRIME_DATASET_FOLDER_PATH, CT_FOLDER_NAME)

    filenames = os.listdir(os.path.join(PRIME_DATASET_FOLDER_PATH, MRI_FOLDER_NAME))
    filenames_translated = [x for x in filenames if "translated" in x ] 

    ct_folder_target_path =  "./DataPlaceholder/MRI"
    mri_folder_target_path = "./DataPlaceholder/MRI"

    for file in filenames_translated:
        source_mri = os.path.join(mri_base_path, file)
        source_ct = os.path.join(ct_base_path, file)
        mri_destination = os.path.join(mri_folder_target_path, file)
        ct_destination = os.path.join(ct_folder_target_path, file)

        with open(source_mri, 'rb') as src, open(mri_destination, 'wb') as dst:
            dst.write(src.read())
        dst.close()
        with open(source_ct, 'rb') as src, open(ct_destination, 'wb') as dst:
            dst.write(src.read())
        dst.close()


In [ ]:
ct_folder_test_path =  "./DataPlaceholder/CT"
mri_folder_test_path = "./DataPlaceholder/MRI"
files = os.listdir("./DataPlaceholder/MRI")

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from Application.ImageTransformation.Transformations import QNetworkFuseSilu, PixelAveraging, CNNFuseSmoothing
from Presentation.Components.Panels.QualityAssesionPanel import QualityHolder

evaluation_dict = {"MRI":[], "CT":[]}

for file in files[:20]:
    ct = cv2.imread(os.path.join(ct_folder_test_path,file))
    mri = cv2.imread(os.path.join(mri_folder_test_path,file))

    ct = cv2.cvtColor(ct, cv2.COLOR_BGR2GRAY)
    mri = cv2.cvtColor(mri, cv2.COLOR_BGR2GRAY)
    
    if ct is None or mri is None:
        continue
    fused = QNetworkFuseSilu(mri, ct)
    
    metric_mri, metric_ct = QualityHolder._GetSSIM(mri,ct, fused[0])
    evaluation_dict["CT"].append(metric_ct)
    evaluation_dict["MRI"].append(metric_mri)
    
eval = pd.DataFrame(evaluation_dict)
eval.to_csv("QNetworkSiluSSIM.csv", index=False)

In [5]:
import cv2
import os
import numpy as np
import pandas as pd
from Application.ImageTransformation.Transformations import QNetworkFuseSilu, PixelAveraging, CNNFuseSmoothing
from Presentation.Components.Panels.QualityAssesionPanel import QualityHolder

evaluation_dict = {"MRI":[], "CT":[]}

for file in files[:20]:
    ct = cv2.imread(os.path.join(ct_folder_test_path,file))
    mri = cv2.imread(os.path.join(mri_folder_test_path,file))

    ct = cv2.cvtColor(ct, cv2.COLOR_BGR2GRAY)
    mri = cv2.cvtColor(mri, cv2.COLOR_BGR2GRAY)
    if ct is None or mri is None:
        continue
    fused = PixelAveraging(mri, ct)
    
    metric_mri, metric_ct = QualityHolder._GetMI(mri,ct, fused[1])
    evaluation_dict["CT"].append(metric_ct)
    evaluation_dict["MRI"].append(metric_mri)
    
eval = pd.DataFrame(evaluation_dict)
eval.to_csv("PixelMI.csv", index=False)

In [4]:
import cv2
import os
import numpy as np
import pandas as pd
from Application.ImageTransformation.Transformations import QNetworkFuseSilu, PixelAveraging, CNNFuseSmoothing
from Presentation.Components.Panels.QualityAssesionPanel import QualityHolder

evaluation_dict = {"MRI":[], "CT":[]}

for file in files[:20]:
    ct = cv2.imread(os.path.join(ct_folder_test_path,file))
    mri = cv2.imread(os.path.join(mri_folder_test_path,file))

    # ct = cv2.cvtColor(ct, cv2.COLOR_BGR2GRAY)
    # mri = cv2.cvtColor(mri, cv2.COLOR_BGR2GRAY)
    
    
    if ct is None or mri is None:
        continue
    fused = CNNFuseSmoothing(mri, ct)
    
    ct = cv2.cvtColor(ct, cv2.COLOR_BGR2GRAY)
    mri = cv2.cvtColor(mri, cv2.COLOR_BGR2GRAY)
    
    metric_mri, metric_ct = QualityHolder._GetEPI(mri,ct,cv2.cvtColor(fused[1], cv2.COLOR_BGR2GRAY))
    evaluation_dict["CT"].append(metric_ct)
    evaluation_dict["MRI"].append(metric_mri)
    
eval = pd.DataFrame(evaluation_dict)
eval.to_csv("CNNSmoothEPIcsv", index=False)